In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Import pandas and store dataframe in a variable

In [ ]:
import pandas as pd
car_filepath='../input/cardataset/data.csv'
car_data=pd.read_csv(car_filepath)

In [ ]:
car_data.head(4)
car_data.describe()

In [ ]:
print(car_data.columns)
car_data = car_data.dropna(axis=0)#drop any missing values
#for i in range (len(car_data.Make))

In [ ]:
feature_col=['Make','Model','Year','Engine HP','Market Category']#selected most important ones
#instead of algo making selection

# Convert String features to numbers
*DecisionTreeRegressor ,its fit method doesnt accept string variables*

In [ ]:
#from sklearn import preprocessing
#le = preprocessing.LabelEncoder()
"""In Future
1-Instead of using encoder I plan to store all Make elements(and Model and Category) in lists
and use 
the position inside list as number,then I can easily convert back any number to list
2-Want to be able to predict for any car ie is not in data,will create a variable and test.
For any car that doesnt have a name that exists in list, ie some unseen car, I would
simply add that car Make to these lists and run my predictor, but only allow to select from 
available catoegories for new elements since, predictor depends on category of car so wont
be able to make predictor for new category."""




carMakeLst=[]#variable will contain list of all car brands that are there in the dataset
for Brand in (car_data.Make):
    if Brand not in carMakeLst:
        carMakeLst.append(Brand)

car_data['MakeInd']=[carMakeLst.index(Make) for Make in car_data.Make]

#print(car_data.MakeInd)

carCategory=[]#variable will contain list of all car categories that are there in the dataset
for Category in (car_data['Market Category']):
    if Category not in carCategory:
        carCategory.append(Category)
#print(carCategory)
car_data['MktCategoryInd']=[carCategory.index(Cat) for Cat in car_data['Market Category']]


carModel=[]#variable will contain list of all car models that are there in the dataset
for Model in (car_data['Model']):
    if Model not in car_data.Model:
        carModel.append(Model)
#print(carModel)
car_data['ModelLstInd']=[carModel.index(Mod) for Mod in car_data['Model']]




"""Make is string was
causing problems with decision treeregression,Market Categ and model also causing problems
So I will convert them to numbers and add new column to dataframe"""
new_feature_col=['MakeInd','ModelLstInd','Year','Engine HP','MktCategoryInd']
X=car_data[new_feature_col]#will use only these features for training and ignore rest
y=car_data.MSRP
#print(y)

# Split Test data and use Decision Tree Regressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(X,y, random_state = 1)#splitting data to
#to accuately evaluate model,otherwise since model was trained on same data so 
#would make accurate predictions of same dataset but might struggle with other data 
#due to overfitting.
car_pred_model= DecisionTreeRegressor(random_state=1)
car_pred_model.fit(train_X,train_y)

# Testing and Evaluating my trained Model

In [ ]:
val_predictions = car_pred_model.predict(val_X)
print(val_predictions[0:10])
print(val_y[0:10])

In [ ]:
from sklearn.metrics import mean_absolute_error
val_mae =(mean_absolute_error(val_y, val_predictions))
avg_price=5.007267e+04
print("Error is: "+str(val_mae*100/avg_price)+"% of avg price")

In [ ]:
print(val_predictions[0:2])
print(val_y[0:2])#ActPrice
print("ActPrice: "+str(car_data.MSRP[4788])+" Make: "+str(car_data.Make[4788])+" Category: "+str(car_data['Market Category'][4788]))
print("ActPrice: "+str(car_data.MSRP[4474])+" Make: "+str(car_data.Make[4474])+" Category: "+str(car_data['Market Category'][4474]))





In [ ]:
#Having some trouble converting back the codes to strings
#eg:lets predict price of the first car ie car_data[0]
myprediction=car_pred_model.predict(X[200:201])
print("myPrice: "+str(myprediction))
print(car_data[200:201])
print("\n \n Difference: "+str(36090-myprediction)+"$")

*Comment:"Got rid of the encoder (which I used to convert string to int because DecisionTreeRegressor fit would only accept numbers) since inverse encoding had problems. "*
* Let's create a car, (ie not a car from dataset but a new car) with some fields that software wont be familiar with, eg a new Model perhaps.  Then we shall see if software can predict its price.


# Testing car Price on a car that is not in dataset
**I have made sure its category already exists in data set since predictions are highly dependent on features like that**

In [ ]:
#print(X[200:201])
#print(carMakeLst)
TestMake=['Nissan']
TestModel=['Sunny']
#print(carModel)
TestCategory=['Performance']
TestEngine_HP=150.0
TestYear=2017
print("Must define these categories for a new car"+str(new_feature_col)+"\n")#categories that I must define for a car,predictor wouldnt work without them
#Now lets convert these to numbers
if 'Performance' in carCategory:
    #print(carCategory.index('Performance'))
    TestCategory=carCategory.index('Performance')#converting to num for predictor
else:
    carCategory.append('Performance')
    TestCategory=carCategory.index('Performance')
    
if 'Nissan' in carMakeLst:
    #print(carMakeLst.index('Nissan'))
    TestMake=carMakeLst.index('Nissan')
else:
    carMakeLst.append('Nissan')
    TestMake=carMakeLst.index('Nissan')

if 'Sunny' in carModel:
    #print(carModel.index('Sunny'))
    TestModel=carModel.index('Sunny')
else:
    carModel.append('Sunny')
    TestModel=carModel.index('Sunny')
    

d = {'MakeInd': TestMake, 'ModelLstInd': [TestModel],'Year':[TestYear],'Engine_HP':[TestEngine_HP],'MktCategory':[TestCategory]}
df = pd.DataFrame(data=d)
SunnyPricePred = car_pred_model.predict(df)
print("Predicted Nissan Sunny Price: "+str(SunnyPricePred))
print("Actual Price is 20000, so error of about: "+str((SunnyPricePred-20000)*100/SunnyPricePred)+"%")

        

